In [1]:
%%javascript
$('#appmode-leave').hide();
$('#copy-binder-link').hide();
$('#visit-repo-link').hide();

<IPython.core.display.Javascript object>

# Water anomalies
Despite its simple molecular structure, water is an exceptionally complicated fluid.
Many of its properties do not follow the trends obeyed by other liquids and are often referred to as water anomalies.
Scientists often report more than 50 _anomalous_ properties of water, and some of the most well-known examples are
1. Water has an unusually high melting point for a molecule of such a low molecular weight.
2. Water has an unusually high boiling point for a molecule of such a low molecular weight.
3. A liquid-liquid transition occurs at about 330 K.
4. Pressure reduces ice's melting point.
5. Cold liquid water has a high density that increases on warming (up to 3.984 °C).

Most of these anomalous behaviours have been explained and there is ample scientific (and non-scientific) literature discussing them. As an example, this website [https://water.lsbu.ac.uk/water/water_anomalies.html](https://water.lsbu.ac.uk/water/water_anomalies.html) will provide a good overview, and plenty of references, on the topic.

In this numerical workshop you will use a computational technique called Molecular Dynamics (MD) to study the how the water density changes with temperature.
MD is one of the most widely used type of atomistic simulations, and it is now routinely used in many research groups to complement experimental studies.

## Molecular dynamics
Molecular dynamics is conceptually very simple, an iterative solution of Newton's equations of motions at the atomic level, but subtly complicated to use for direct quantitative comparison with experiments.
Although a detailed description of MD is beyond the scope of this laboratory, it is worth discussing some basic ideas for you to start appreciating the power and limitations of this technique. There are plenty of webpage and tutorials that describe the working principles of MD; Wikipedia has a fairly good an general overview of this topic [https://en.wikipedia.org/wiki/Molecular_dynamics]( https://en.wikipedia.org/wiki/Molecular_dynamics)

In MD the atoms are treated as point particles with a mass and a partial charge. Their interactions are described by simple empirical equations, such as the Coulomb and the van der Waals  (dispersion) forces, supplemented with two-, three- or four-body interactions to better capture the covalent nature of the intramolecular bonds.
For example, in classical molecular dynamics the interaction *energy* between two non-bonded atoms separated by a distance, $r$, can be written as

\begin{equation}
U_{ij} = \frac{1}{4\pi\varepsilon_0}\frac{q_i q_j}{r} + \frac{A}{r^{12}} - \frac{B}{r^6} \tag{1}
\end{equation}

Where the first term is the Coulomb interaction and the last two the repulsive and attractive parts of the van der Waals interactions.
On the other hand the bonded two-, three- and four-body interactions between covalently bonded atoms are typically described by *harmonic* potentials

\begin{eqnarray}
U_{ij}^b &=& K_b(b_{ij}-b_0)^2 \tag{2} \\
U_{ijk}^a &=& K_\theta(\theta_{ijk}-\theta_0)^2 \tag{3} \\
U_{ijkl}^t &=& K_\phi[1+\cos(n\phi_{jikl}-\phi_0)]^2 \tag{3} \\
\end{eqnarray}

where $b_{ij}$, $\theta_{ijk}$ and $\phi_{jikl}$ are the bond lengths, angle and torsional angle between the atoms and the other quantities are fitting parameters, which are key to determine the accuracy of the simulations.

Once the interaction energy is known we can then compute the forces on the atoms as the sum of all pair-wise interactions

\begin{equation}
F_i = \sum_{j\neq i} F_{ij} = -\Bigg[
  \sum \frac{\partial U_{ij}}{\partial x_i} +
  \sum \frac{\partial U_{ij}^a}{\partial x_i} +
  \sum \frac{\partial U_{ijk}^b}{\partial x_i} +
  \sum \frac{\partial U_{ijkl}^t}{\partial x_i} \Bigg] \tag{5} 
\end{equation}

Then, by knowing the positions, velocities and forces for all the atoms at a certain time $t$, we can use the Newton's equations of motions to *predict* the positions and velocities of the particles after a certain (short) amount of time as passed

\begin{eqnarray}
a_i(t) &=& \frac{F_i(t)}{m_i} \tag{6} \\
v_i(t+\delta t) &=& v_i(t) + a_i(t)\delta t \tag{7} \\
x_i(t+\delta t) &=& x_i(t) + v_i(t)\delta t + \frac{1}{2}a_i(t)\delta t^2 \tag{8} \\
\end{eqnarray}

where $a_i$, $v_i$ and $x_i$ are the acceleration, velocity and position of particle $i$, and $\delta t$ is called the time step.
These three equations (or some variants of them) are usually called **equations of motions**.
Now that we have the new atomic positions we can compute the new forces on the atoms, and use again the Newton's equations of motions to *propagate* the atoms' positions further. 
This iterative procedure will generate a **trajectory** for the atoms, and by using energies and velocities collected along the way we will also get information about the temperature, pressure and other thermodynamic quantities of the system.


### Importance of the time step
The time step is one of the most important quantities in MD, and it is key to understand the potentials and limitations of atomistic molecular dynamics simulations.
In fact, for the above equations of motions to be valid, the time step has to be short enough to describe the fastest **atomic** motion in the system, which in the case of water is the O-H stretching mode.
The O-H stretching has a vibrational frequency of approximately $1\times10^{14}$Hz, *i.e* it takes about $1\times10^{-14}$s to complete one oscillation. Therefore, if we want to describe this very fast atomic motion using discrete points in time we would need 10-20 snapshots. Hence the time step has to be of the order of 1~fs ($1\times10^{-15}$s) or less.

Now, let’s imagine running a simulation with a 1 fs time step and that the computer take 10 ms to calculate energies, forces and do one cycle of the equations of motions.
In the table below you can see how long it would you take to simulate a chemical or physical process depending on the time scale it experimentally occurs

| Experimental time scale <img width=50/> | Phenomenon <img width=200/>  | Simulation time <img width=150/>
| :-----: | :--------: |:---------
| 10 fs   | O-H vibration                     | 0.1 s
| 1 ps    | H-bond persistence                | 10 s
| 1 ns    | Ion permeation through a membrane | 3 hours
| 1$\mu$s | Conformational rearrangement      | 115 days 
| 1 ms    | Protein folding (fast)            | 317 days
| 1 s     | Protein folding (typical)         | 317,000 days

Obviously, the time required to do one MD cycle depends on the number of operations the computer has to perform, hence it increases with the system size.
Although computational power has increased exponentially since MD was first introduced in the 1940s and we can now afford to study systems of millions of atoms or of hundreds of nm in size, there are still strong limitations to what can be reliably simulated due to the finite (small) number of atoms is included in the system (compared to Avogadro's number) and the short time scale that the simulation can span.

### Ensemble
MD codes are more complicated that a simple iterative solution of Newton's equations of motions and they include algorithms to control the temperature, pressure and other thermodynamics quantities of the system.
Of particular relevance for this experience is the need to use **thermostats** and **barostats** to fix the boundary conditions of the simulations.
For this laboratory is not important to know the working details of these algorithms, but is key that you are aware that the variables relating to the temperature and pressure of the simulations are input parameters that may need to be changed.

## Scope of the laboratory
The scope of this virtual experiment is to introduce you to atomistic molecular dynamics simulations and to compute the variations of the water density as a function of temperature, and to compare it with experimental values. As briefly mentioned above, the accuracy of the simulation depends on the parameters that are used to compute the intermolecular interactions. 
The water models available for this laboratory are
* SPC/E
* TIP3P
* TIP4P/ew
* TIP5P

which is a very small selections of the available models for water.
The models are listed in increasing level of complexity and one would reasonably expect that the more complex model gives more accurate results.

In this laboratory, you will choose one water model and run a series of simulations to compute the water density at various temperatures and determine the volumetric thermal expansion coefficient of water $\alpha$, which is defined as

\begin{equation*}
\alpha(T) = \frac{1}{V}\bigg(\frac{\mathrm{d}V}{\mathrm{d}T}\bigg)_P = \bigg(\frac{\mathrm{d}\ln V}{\mathrm{d}T}\bigg)_P \tag{9}
\end{equation*}

where the subscript _P_ means that the simulations are performed at constant pressure, and we have explicitly indicated the the volumetric thermal expansion coefficient depends on the temperature. In this laboratory we will assume that the temperature dependence of $\alpha(T)$ can be described with a simple polynomial expansion, which means that $\ln V(T)$ can also be described with a polynomial expansion

\begin{eqnarray*}
\ln V(T) &=& A + Bx + Cx^2 + \dots \\
\frac{\mathrm{d}\ln V}{\mathrm{d}T} = \alpha(T) &=& B + 2Cx + \dots
\end{eqnarray*}

From each simulation you will compute the average volume of you simulation cell, discarding the initial portion of the simulation, where the system is out of equilibrium. 

You will then fit the logarithm of the average volumes _vs_ time with a polynomial function, and compute the value of its derivative, $\alpha$, at a few selected temperatures.

## Your first MD simulation

You will now run your first MD simulation.
In the next Jupyter Notebook we will go under the hood of a molecular dynamics code in written python to see how the ensemble parameters are passed to the code, how to modify them and what outputs the code produces.

This first simulation will also give you an indication of how long a typical run takes. Please note that the timing depends on the load of the machine, so the more users are working on this laboratory to slower the calculations might get. This could be the "worst case scenario" given that all the class would be accessing the VM.

We will now exit from the "appMode" and look at a MD code in python.

## Final report
### The introduction should outline
1. The scope of the laboratory
2. The water model that you have chosen
3. The set of temperatures that you have chosen
4. The length of the simulations

### The results section should show (or discuss)
1. How you computed the average, standard error and standard deviation of the volume/density
2. A table with the average (molar) volumes and densities
3. A plot of $\ln V$ _vs_ T and the polynomial fitting function
4. The volumetric thermal expansion coefficient of water at 20$^\circ$C and 50$^\circ$C

### The discussion section should contain 
1. A comparison between your values for the density and the thermal expansion coefficient with literature values for "real" water and obtained from other simulations with the same water model.

## Optional work - density maximum
1. Does the water model that you have chosen show a maximum in the water density? 
2. If yes how does that compare with the experimental value

## Launch virtual experiment
Let's now have a look at the python notebook to run MD simulations
[Molecular dynamics Simulation](md.ipynb)